In [11]:
from Models.BlackBox.lmart.lgbmranker_class import LGBMRanker_class
import pandas as pd
%reload_ext autoreload
%autoreload 2

In [12]:
lMart_parameter = dict(
    boosting_type=["dart", "gbdt"],
    num_leaves=[2, 5, 10, 20, 30, 40],
    max_depth=[-1],
    n_estimators=[75, 80, 100, 150, 200],
    learning_rate=[0.02, 0.05, 0.08, 0.1, 0.15, 0.2],
    reg_lambda=[0.00005, 0.0001, 0.0002, 0.0003]
)
gridsearch_parameters = dict(
    path="./../../../outputs/scores/",
    name="0",
    nDCG_at=15
)

# Grid-search

In [13]:
gs = LGBMRanker_class(**gridsearch_parameters)

In [14]:
best_ = gs.grid_search(lMart_parameter)
gs.save_model(best_[0],"LGBMRanker")

Finding the best model: 100%|██████████| 1440/1440 [14:55<00:00,  1.61it/s, nDCG_15=0.986]


# Metric evaluation

In [15]:
best_model = gs.load_model("./saved_models/LGBMRanker")

In [16]:
# nDCG
nDCG_train = gs.eval_model(model=best_model, dt=gs.train, qIds=gs.qIds_train, nDCG_at=[1,10,15])
nDCG_valid = gs.eval_model(model=best_model, dt=gs.valid, qIds=gs.qIds_valid, nDCG_at=[1,10,15])
nDCG_test = gs.eval_model(model=best_model, dt=gs.test, qIds=gs.qIds_test, nDCG_at=[1,10,15])

other_evals_tr = gs.other_eval(df=gs.test, qIds=gs.qIds_test, nDCG_at=[1,10,15])

display(pd.DataFrame([nDCG_train,nDCG_valid,nDCG_test],index=["Training","Validation","Test"]))

print("Test set")
display(pd.DataFrame(other_evals_tr).T)

,nDCG@1,nDCG@10,nDCG@15
Training,0.9652,0.9773,0.9804
Validation,0.9594,0.9803,0.9863
Test,0.9394,0.9697,0.9769


Test set


,nDCG@1,nDCG@10,nDCG@15
random_permutation,0.6759,0.7575,0.7946
perfect_nDCG,1.0000,1.0000,1.0000
worste_nDCG,0.2345,0.4655,0.5656
